# MATH 300: Numerical Analysis I Recitation

## Instructor: Liam Doherty

## Meeting Time/Place: W 11:00AM, Curtis 344

### Office Availability: MRC hours MTR 5p-7p (M 5-7 & T 5-6 FTF, others online), or in Korman 257 by appointment

We'll start with some Julia basics. Let's first have a look at some resources available for the language:

Julia main page: www.julialang.org

Documentation: https://docs.julialang.org/en/v1/

Discourse: https://discourse.julialang.org/

Some docs pages for common packages:

* Plots: http://docs.juliaplots.org/latest/

* Linear Algebra: https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/

* Differential Equations: https://diffeq.sciml.ai/stable/

* Statistics: https://docs.julialang.org/en/v1/stdlib/Statistics/

I expect that we will only need the Plots (and maybe LinearAlgebra) packages for most programs in this course. To add a package that you do not have downloaded, navigate to the REPL (in the terminal area with Atom or VS Code) and type

] add [PackageName]

The ] at the beginning brings you to the package manager. Example in Atom.

Once you do this, you should be able to use your packages in code like so:

In [1]:
using Plots

If you've made a typo, or you don't have the package installed, you'll get a message like

In [2]:
using BadPackage

LoadError: ArgumentError: Package BadPackage not found in current path:
- Run `import Pkg; Pkg.add("BadPackage")` to install the BadPackage package.


If you get this, try installing [BadPackage] using the method above. If you've spelled it right and the package exists and is functional, retrying the above command (using BadPackage) should work just fine. Now that we're able to navigate around the IDE and install packages, let's look at some different types of variables that will show up in code:

In [3]:
name = "Liam Doherty" # Strings
integer = 10 # Integer number
float = 2.7 # Floating Point number
integer_as_float = 1. # Integer value, stored as floating point (useful for type stability)

1.0

You can suppress outputs with semicolons:

In [5]:
variable = 4;

If we're unsure of the type of a variable, we can use typeof() to check it!

In [6]:
typeof(integer_as_float)

Float64

If we want to print something (i.e., outside of Jupyter notebooks or if we want to print more than just the last result in Jupyter notebooks), we can use println:

In [80]:
println(name)
println(float)

Liam Doherty
2.7


Technically, print works as well, but println adds a line break, which is often convenient (and is what Python's print statement does by default).

If we ever want to look at documentation on a particular function (but don't want to go searching online), we can do that with the ? prefix:

In [24]:
? println

search: println printstyled print sprint isprint parametric_colorant



```
println([io::IO], xs...)
```

Print (using [`print`](@ref)) `xs` followed by a newline. If `io` is not supplied, prints to [`stdout`](@ref).

# Examples

```jldoctest
julia> println("Hello, world")
Hello, world

julia> io = IOBuffer();

julia> println(io, "Hello, world")

julia> String(take!(io))
"Hello, world\n"
```


Now, let's take a look at one of the most important structures in Julia that will be in almost all code: functions. The basic syntax is

In [9]:
function function_name(input)
    # Do stuff to shift input to output
    output = input
    return output
end

function_name (generic function with 1 method)

Notice a few things:

The function name is all lowercase and uses underscores for word separation. This is in contrast to the package names like LinearAlgebra which use camel case.

Another thing to notice is that there is a return statement. Please use it! Julia technically does not require it, and will return the last value (i.e., if I just wrote output instead of return output), but for purposes of readability the return statement is very useful.

Third, notice the indentation. While not strictly required (unlike Python), indentation is critical to writing good, readable code. While there are ends for statements/functions/etc, it is much easier to keep track of nesting structure when there is proper indentation. Every time you have a new block of code requiring an end, indent one more time. 

Finally, notice the function ... end syntax. This should be very familiar to you if you've used MATLAB. The end statement is all over the place (functions, for loops, conditional statements, etc.), so if you are primarily a Python programmer, pay attention and don't forget them!

You can also write (small) functions inline (think of these as easy-to-write mathematical functions like x^2):

In [10]:
f(x) = x^2

f (generic function with 1 method)

In [11]:
f(4)

16

If you want, you can also set defaults in your function's inputs, or use keyword arguments:

In [76]:
function calculate_gravity_force_earth(mass, g = 9.8; return_force = true)
    if return_force == true
        return mass*g
    else
        return -1.
    end
end

calculate_gravity_force_earth (generic function with 2 methods)

In [77]:
calculate_gravity_force_earth(10)

98.0

In [78]:
calculate_gravity_force_earth(10, return_force = false)

-1.0

A very common data structure that is used in numerical analysis codes is the matrix (vector). Let's look at some basic syntax surrounding them:

In [25]:
A = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [26]:
V = [1 2 3 4]

1×4 Matrix{Int64}:
 1  2  3  4

In [27]:
VT = [1; 2; 3; 4] # Transpose of V, often we want column vectors. Also, notice that this is a vector, not a matrix!

4-element Vector{Int64}:
 1
 2
 3
 4

In [28]:
V' # We can also take the transpose directly with ', but again notice the type difference!

4×1 adjoint(::Matrix{Int64}) with eltype Int64:
 1
 2
 3
 4

One thing to note is that Julia uses 1-based indexing. That is, indexing arrays/vectors starts at 1, not 0:

In [35]:
println(V[1])

1


We can also slice arrays to obtain pieces of them without mutilating the original array:

In [37]:
B = [1 2 3; 4 5 6; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [38]:
B[:, 3]

3-element Vector{Int64}:
 3
 6
 9

In [82]:
B[1:2, 2:end] # Notice the use of end here!

2×2 Matrix{Int64}:
 2  3
 5  6

Another important distinction to make with Julia from, say, Python, is that it is a column-major language. What this means is if we were to "flatten" an array, it would read column by column instead of row by row. Example:

In [44]:
for i = 1:length(B)
    print("$(B[i]) ")
end

1 4 7 2 5 8 3 6 9 

This may have implications in, for example, loops where you are indexing over an array. It isn't common, but it's good to have in the back of your mind when debugging (especially since Julia won't throw an error for something like this, but your output will still be garbage! A good practice is to check your output and make sure it is what you "expect").

Now we can take a look at some control flow (there are some examples above too)!

In [65]:
for i = 1:5
    print("$i ")
end

println() # Line break in output

j = 1
while j <= 5
    print("$j ")
    j += 1
end

println()

k = 100
if k > 50
    println("k is larger than 50!")
elseif k > 25
    println("k is greater than 25!")
else
    println("k is less than 25.")
end

1 2 3 4 5 
1 2 3 4 5 
k is larger than 50!


Two statements that are useful in loops are continue and break. Continue skips the rest of the code in the outer block and goes to the next iterate (i.e., starts back at the top), and break immediately jumps out of the loop entirely.

In [55]:
i = 1
while true
    if i < 10
        i += 1
        continue
    elseif i > 20
        break
    else
        println(i)
        i += 1
    end
end

10
11
12
13
14
15
16
17
18
19
20


Another important concept is that of broadcasting. This will likely appear frequently in your Julia code. Let's say you have a function that takes an integer (notice the type specification) and multiplies it by 6:

In [56]:
function multiply_by_6(n::Int)
   return 6*n
end

multiply_by_6 (generic function with 1 method)

What happens if we want to apply this function to an entire array? Let's see what happens.

In [57]:
array = [1 2 3]

1×3 Matrix{Int64}:
 1  2  3

In [58]:
multiply_by_6(array)

LoadError: MethodError: no method matching multiply_by_6(::Matrix{Int64})
[0mClosest candidates are:
[0m  multiply_by_6([91m::Int64[39m) at In[56]:1

We have an issue, since the function is designed to take in a single integer, not a list of them! To fix this, we can use broadcasting. Syntactically, all we need to do is put a simple "." operator.

In [83]:
@. multiply_by_6(array)

1×3 Matrix{Int64}:
 6  12  18

This will (more or less) unpack the array, feed each element into the function, and then package the output in the same way that the input was provided. This is very helpful in many scenarios where we want to evaluate a function many times (maybe at several timesteps, etc.) and we can do it easily with this notation! Additionally, if it is awkward to use this kind of notation but you still want to perform operations on an entire array at once, you can use the "." macro:

In [75]:
xs = LinRange(-1, 1, 100)
xs = convert(Vector, xs)

xs .+= 2 # One way of doing this is to use the usual dot in front of the operator, but this looks a bit awkward
println("$(xs[1]), $(xs[end])")

@. xs += 2 # Using the "." macro (macros have @ in front of them, and there are many macros with many different uses)
println("$(xs[1]), $(xs[end])")

1.0, 3.0
3.0, 5.0


For the ambitious among you, there are many more features of Julia's core, not even considering the rich ecosystem of packages that are available! Julia's documentation (link at the top of this notebook) gives all that you need to be able to learn the intricacies of the base language, with more advanced concepts that are more Julia-specific (as opposed to the syntax here, where most of the above is in some way present in other languages). Give the docs a look, or even jump into learning the most common packages, like Plots and DifferentialEquations, or even have some fun with more advanced packages like Flux (deep learning) or Turing (probabilistic programming) or many of the others!